In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.master("local").appName("stock-price-streaming") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,org.mongodb.spark:mongo-spark-connector_2.11:2.4.0") \
        .getOrCreate())


In [24]:
# # Consumer for Google:

# google_df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092") \
#             .option("subscribe", "google-topic").load()

# # Convert the data back to string:
# google_df_string = google_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# # Create a temporary view:
# google_df_string.createOrReplaceTempView("google_message")

# # Print the output in the console:
# google_res = spark.sql("Select * from google_message")
# google_res.writeStream.format("console").outputMode("append").start()


In [22]:
companies = ["amazon", "apple", "coke", "google", "ibm", "johnson", "meta", "nvidia", "pinterest", "tesla"]

In [17]:
def consumer_func(company ):
    '''
    This is a consumer function that will take in the company name that will subscribe to the topic
    of that particular topic.
    '''
    dataframe = f"{company}" + "_" + "df"
    dataframe_string = f"{company}" + "_" +"df" + "_" + "string"
    topic_name = f"{company}" + "-" + "topic"
    temp_table = f"{company}" + "_" + "message"
    temp_res = f"{company}" + "_" +"res"
    
    
    # Subscribe to the topic:
    dataframe = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092") \
                .option("subscribe", f"{topic_name}").load()
    
    # Convert the recieved data from bytes to string:
    dataframe_string = dataframe.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
    
    # Create a temporary view:
    dataframe_string.createOrReplaceTempView(f"{temp_table}")
    
    # Print the output in the console:
    temp_res = spark.sql(f"select * from {temp_table}")
    temp_res.writeStream.format("console").outputMode("append").start()

In [23]:
for company in companies:
    consumer_func(company=company)